# XOR Multiple Inputs/Targets

Testing...

In [1]:
from conx import Network, Layer, SGD

Using Theano backend.


In [8]:
net = Network("XOR", 2, 2, 1, activation="sigmoid")
net.connect()
dataset = [
    ([0, 0], [0]),
    ([0, 1], [1]),
    ([1, 0], [1]),
    ([1, 1], [0])
]

In [9]:
net.set_dataset(dataset)

Split dataset into:
   4 train inputs
   0 test inputs
Input Summary:
   length  : 4
   training: 4
   testing : 0
   shape  : (2,)
   range  : (0.0, 1.0)
Target Summary:
   length  : 4
   training: 4
   testing : 0
   shape  : (1,)
   range  : (0.0, 1.0)


In [10]:
net.get_target(0)

[0.0]

In [11]:
net.compile(loss='mean_squared_error',
            optimizer=SGD(lr=0.3, momentum=0.9))

In [12]:
net.config["image_maxdim"] = 50

In [13]:
net.build_widget()

<IPython.core.display.Javascript object>

In [15]:
net.train(epochs=2000, accuracy=1.0, report_rate=25)

Training...
Epoch # 2025 | train error 0.12532 | train accuracy 0.50000 | validate% 0.50000
Epoch # 2050 | train error 0.12531 | train accuracy 0.50000 | validate% 0.50000
Epoch # 2075 | train error 0.12531 | train accuracy 0.50000 | validate% 0.50000
Epoch # 2100 | train error 0.12530 | train accuracy 0.50000 | validate% 0.50000
Epoch # 2125 | train error 0.12530 | train accuracy 0.50000 | validate% 0.50000
Epoch # 2150 | train error 0.12530 | train accuracy 0.50000 | validate% 0.50000
Epoch # 2175 | train error 0.12529 | train accuracy 0.50000 | validate% 0.50000
Epoch # 2200 | train error 0.12529 | train accuracy 0.50000 | validate% 0.50000
Epoch # 2225 | train error 0.12529 | train accuracy 0.50000 | validate% 0.50000
Epoch # 2250 | train error 0.12528 | train accuracy 0.50000 | validate% 0.50000
Epoch # 2275 | train error 0.12528 | train accuracy 0.50000 | validate% 0.50000
Epoch # 2300 | train error 0.12527 | train accuracy 0.50000 | validate% 0.50000
Epoch # 2325 | train error 0

In [16]:
net.test()

Testing on training dataset...
# | inputs | targets | outputs | result
0 | [0.0, 0.0] | [0.0] | [0.0] | correct
1 | [0.0, 1.0] | [1.0] | [1.0] | correct
2 | [1.0, 0.0] | [1.0] | [0.5] | X
3 | [1.0, 1.0] | [0.0] | [0.5] | X
Total count: 4
Total percentage correct: 0.5


In [17]:
net.propagate_to("input", [0, 1])

[0.0, 1.0]

In [18]:
net.propagate([0.5, 0.5])

[0.50176251]

In [19]:
net.propagate_to("hidden", [1, 0])

[3.1880122151051182e-06, 0.00011603038001339883]

In [20]:
net.propagate_to("output", [0.25, 0.75])

[0.6228697299957275]

In [21]:
net.propagate_to("input", [0.25, 0.25])

[0.25, 0.25]

In [22]:
net.propagate_from("input", [1.0, 1.0])

[0.41396368]

In [24]:
net.propagate_from("hidden", [1.0, 0.0])

[0.51091605]

In [25]:
net.test()

Testing on training dataset...
# | inputs | targets | outputs | result
0 | [0.0, 0.0] | [0.0] | [0.0] | correct
1 | [0.0, 1.0] | [1.0] | [1.0] | correct
2 | [1.0, 0.0] | [1.0] | [0.5] | X
3 | [1.0, 1.0] | [0.0] | [0.5] | X
Total count: 4
Total percentage correct: 0.5


In [26]:
from conx import Network, Layer, SGD

In [27]:
net = Network("XOR2")
net.add(Layer("input1", shape=1))
net.add(Layer("input2", shape=1))
net.add(Layer("hidden1", shape=10, activation="sigmoid"))
net.add(Layer("hidden2", shape=10, activation="sigmoid"))
net.add(Layer("shared-hidden", shape=5, activation="sigmoid"))
net.add(Layer("output1", shape=1, activation="sigmoid"))
net.add(Layer("output2", shape=1, activation="sigmoid"))

In [28]:
net

<Network name='XOR2'>

In [29]:
net.connect("input1", "hidden1")
net.connect("input2", "hidden2")
net.connect("hidden1", "shared-hidden")
net.connect("hidden2", "shared-hidden")
net.connect("shared-hidden", "output1")
net.connect("shared-hidden", "output2")

In [30]:
net.set_input_layer_order("input1", "input2")
net.set_output_layer_order("output1", "output2")

In [31]:
net.compile(loss='mean_squared_error',
            optimizer=SGD(lr=0.3, momentum=0.9))

In [32]:
dataset = [
    ([[0],[0]], [[0],[0]]),
    ([[0],[1]], [[1],[1]]),
    ([[1],[0]], [[1],[1]]),
    ([[1],[1]], [[0],[0]])
]

In [33]:
net.set_dataset(dataset)

Split dataset into:
   4 train inputs
   0 test inputs
Input Summary:
   length  : 4
   training: 4
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)
Target Summary:
   length  : 4
   training: 4
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)


In [37]:
net.config["image_maxdim"] = 50

In [38]:
net.build_widget()

<IPython.core.display.Javascript object>

In [39]:
net["shared-hidden"].colormap = "spring"
net.propagate_from("hidden1", [0.0, 1.0] * 5)

[[0.50633931], [0.6151036]]

In [40]:
net["shared-hidden"].colormap = "hot"
net.propagate_from("hidden1", [1.0, 1.0] * 5, ["shared-hidden"])

[0.94672239, 0.95694375, 0.28714341, 0.19378476, 0.52984357]

In [41]:
net.propagate_to("output1", [[1], [0]])

[0.6395641565322876]

In [42]:
for i in range(20):
    (epoch_count, loss, acc, val_acc) = net.train(epochs=100, verbose=0)
    for index in range(4):
        net.propagate(dataset[index][0])

In [13]:
net.train(epochs=2000, accuracy=1.0, report_rate=25)

Training...
Epoch #   25 | train loss 0.50041 | train acc 1.00000 | validate% 0.00000
Epoch #   50 | train loss 0.50024 | train acc 1.00000 | validate% 0.00000
Epoch #   75 | train loss 0.50011 | train acc 1.00000 | validate% 0.00000
Epoch #  100 | train loss 0.50007 | train acc 1.00000 | validate% 0.00000
Epoch #  125 | train loss 0.50004 | train acc 1.00000 | validate% 0.00000
Epoch #  150 | train loss 0.50001 | train acc 1.00000 | validate% 0.00000
Epoch #  175 | train loss 0.49998 | train acc 1.00000 | validate% 0.00000
Epoch #  200 | train loss 0.49995 | train acc 1.00000 | validate% 0.00000
Epoch #  225 | train loss 0.49992 | train acc 1.00000 | validate% 0.00000
Epoch #  250 | train loss 0.49988 | train acc 1.00000 | validate% 0.00000
Epoch #  275 | train loss 0.49983 | train acc 1.00000 | validate% 0.00000
Epoch #  300 | train loss 0.49976 | train acc 1.00000 | validate% 0.00000
Epoch #  325 | train loss 0.49967 | train acc 1.00000 | validate% 0.00000
Epoch #  350 | train loss 

In [14]:
net.propagate_from("input1", [0.0])

[[0.5], [0.5]]

In [15]:
net.propagate_from("shared-hidden", [0.0] * 5)

[[0.5], [0.5]]

In [16]:
net.propagate_to("hidden1", [[1], [1]])

[0.792599081993103,
 0.782215416431427,
 0.84234219789505,
 0.1320749670267105,
 0.1610153168439865,
 0.1744602918624878,
 0.4586453139781952,
 0.1609877198934555,
 0.3683990240097046,
 0.15291506052017212]

In [43]:
net.test()

Testing on training dataset...
# | inputs | targets | outputs | result
0 | [[0.0], [0.0]] | [ 0.  0.] | [ 0.02250661  0.02171772] | correct
1 | [[0.0], [1.0]] | [ 1.  1.] | [ 0.97696429  0.97674233] | correct
2 | [[1.0], [0.0]] | [ 1.  1.] | [ 0.97722149  0.97841221] | correct
3 | [[1.0], [1.0]] | [ 0.  0.] | [ 0.02640889  0.02666857] | correct
Total count: 4
Total percentage correct: 1.0


In [18]:
net.slice_dataset(2)

Slicing dataset 0:2...
Input Summary:
   length  : 2
   training: 2
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)
Target Summary:
   length  : 2
   training: 2
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)


In [19]:
net.train(epochs=2000, accuracy=1.0, report_rate=25)

Training...
Epoch #  687 | train loss 0.01742 | train acc 2.00000 | validate% 1.00000


# Conx model is a Keras Model

In [44]:
from keras.utils.vis_utils import model_to_dot
from IPython.display import HTML

In [45]:
dot = model_to_dot(net.model, rankdir="BT")

In [46]:
HTML(dot.create_svg().decode())